<a href="https://colab.research.google.com/github/mkielo3/inekf_demo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Library for Lie Groups
!git clone -q https://github.com/decargroup/pymlg.git
!pip install -q -e ./pymlg

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pymlg (pyproject.toml) ... done


In [1]:
import pandas as pd

1.30923250956986e-05

In [29]:
import matplotlib.pyplot as plt


In [32]:
from tests import test

In [33]:
test()

AssertionError: 

In [27]:
reference_df

,timestamp,object,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z,rot_11,rot_12,rot_13,rot_21,rot_22,rot_23,rot_31,rot_32,rot_33
0,0.0045,robot,0.000000e+00,0.000000e+00,-0.000099,0.000000,0.000000,-0.044145,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000
1,0.0050,robot,-2.359784e-07,-1.723725e-08,-0.000123,-0.000944,-0.000069,-0.050085,1.000000,0.000002,0.000020,0.000002,-1.000000,0.000009,0.000020,-0.000009,-1.000000
2,0.0055,robot,-9.439083e-07,-6.894706e-08,-0.000149,-0.001888,-0.000138,-0.056024,1.000000,0.000004,0.000040,0.000004,-1.000000,0.000019,0.000040,-0.000019,-1.000000
3,0.0060,robot,-2.123779e-06,-1.551255e-07,-0.000179,-0.002832,-0.000207,-0.061964,1.000000,0.000006,0.000060,0.000006,-1.000000,0.000028,0.000060,-0.000028,-1.000000
4,0.0065,robot,-3.827179e-06,-2.755959e-07,-0.000211,-0.003982,-0.000275,-0.068164,1.000000,0.000010,0.000085,0.000010,-1.000000,0.000037,0.000085,-0.000037,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40287,9.9995,landmark_2,9.860497e-01,1.014279e+00,-0.463278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40288,10.0000,robot,2.401317e+00,9.557897e-03,0.825330,0.154984,0.012640,0.031690,0.994784,0.091141,-0.045803,0.095805,-0.988985,0.112826,-0.035015,-0.116626,-0.992558
40289,10.0000,landmark_2,9.860497e-01,1.014279e+00,-0.463278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40290,10.0000,robot,2.401057e+00,9.435155e-03,0.825449,0.154480,0.012310,0.031828,0.994782,0.091157,-0.045809,0.095824,-0.988972,0.112924,-0.035010,-0.116724,-0.992547
